In [ ]:
# In[1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# %%
# define function
import src.SAT_function as data_process
import src.Data_Preprocess as preprocess

In [ ]:
# import src.slurm_cluster as scluster
# client, scluster = scluster.init_dask_slurm_cluster()

In [ ]:
def func_mk(x):
    """
    Mann-Kendall test for trend
    """
    results = data_process.mk_test(x)
    slope = results[0]
    p_val = results[1]
    return slope, p_val
 

In [ ]:
# Input the MMEM annual mean SAT data
input_model = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Datafiles/'

# CanESM_data   = xr.open_mfdataset(input_model + 'tas_CanESM5_annual_ano_1850_2022.nc',chunks = {'run':1})
# IPSL_data     = xr.open_mfdataset(input_model + 'tas_IPSL_annual_ano_1850_2022.nc',chunks = {'run':1})
# EC_Earth_data = xr.open_mfdataset(input_model + 'tas_EC_Earth_annual_ano_1850_2022.nc',chunks = {'run':1})
ACCESS_data   = xr.open_mfdataset(input_model + 'tas_ACCESS_annual_ano_1850_2022.nc',chunks = {'run':1})
# MPI_ESM_data  = xr.open_mfdataset(input_model + 'tas_MPI_ESM_annual_ano_1850_2022.nc',chunks = {'run':1})
# MIROC_data    = xr.open_mfdataset(input_model + 'tas_MIROC6_annual_ano_1850_2022.nc',chunks = {'run':1})
# MMEM_annual_data = xr.open_mfdataset(input_model + 'tas_MMEM_annual_anomalies_ds.nc',chunks = {'lat':10,'lon':10})

In [ ]:
ACCESS_data

## Calculate the ensemble mean trend pattern during 1950-2022

### Apply the MMEs GSAT timeseries to extract forced and unforced component of GSAT

In [ ]:
# input GSAT time series
input_ts_dir = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Supp_Figure1/'

GSAT_ACCESS_annual = xr.open_dataset(input_ts_dir + 'GMSAT_ACCESS_annual_timeseries_ENS.nc')
GSAT_ACCESS_annual

In [ ]:
ACCESS_data['tas'].values.shape

In [ ]:
slope_Forced_signal, intercept_Forced_signal = data_process.linear_regression_single_gmst_multi_sat(
    GSAT_ACCESS_annual['tas'],
    ACCESS_data['tas'].values)

In [ ]:
# separate the GSAT into forced and internal variability
lat = ACCESS_data['lat']
lon = ACCESS_data['lon']
run = ACCESS_data['run']

In [ ]:
slope_Forced_signal.shape

In [ ]:
# check the first run
slope_Forced_signal[0,:,:]

In [ ]:
slope_Forced_signal[1,:,:]

### Check whether each run's slope is different

In [ ]:
slope_Forced_signal_da = xr.DataArray(slope_Forced_signal, coords=[run,lat, lon], dims=['run','lat', 'lon'])
intercept_Forced_signal_da = xr.DataArray(intercept_Forced_signal, coords=[run, lat, lon], dims=['run','lat', 'lon'])

In [ ]:
slope_Forced_signal_da

In [ ]:
# reconstruct the forced signal
GSAT_ACCESS_Forced_signal = GSAT_ACCESS_annual['tas'] * slope_Forced_signal_da  + intercept_Forced_signal_da

In [ ]:
GSAT_ACCESS_Forced_signal

In [ ]:
# reorder the dimensions
GSAT_ACCESS_Forced_signal = GSAT_ACCESS_Forced_signal.transpose('run','year','lat','lon') 

In [ ]:
GSAT_ACCESS_Forced_signal

In [ ]:
GSAT_ACCESS_Forced_signal[0,0,:,:]

In [ ]:
GSAT_ACCESS_Forced_signal[1,0,:,:]

In [ ]:
# calculate the internal variability
GSAT_ACCESS_Internal_variability = ACCESS_data - GSAT_ACCESS_Forced_signal

In [ ]:
print(GSAT_ACCESS_Internal_variability['tas'][0,0,:,:].values)

In [ ]:
print(GSAT_ACCESS_Internal_variability['tas'][1,0,:,:].values)

In [ ]:
GSAT_ACCESS_Internal_variability['tas'].min().values

In [ ]:
# output the forced and unforced anomalies
output_dir = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure2/ACCESS/'

GSAT_ACCESS_Forced_signal.to_netcdf(output_dir + 'GSAT_ACCESS_Forced_anomalies_1850_2022.nc')
GSAT_ACCESS_Internal_variability.to_netcdf(output_dir + 'GSAT_ACCESS_Internal_Variability_anomalies_1850_2022.nc')